In [1]:
import pandas as pd
import pm4py
from pm4py.objects.conversion.log.versions import to_dataframe
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pm4py.objects.log.importer import xes

In [3]:
from pm4py.objects.log.importer.xes import importer as xes_importer

### Таблицы

In [4]:
domestic_declarations = xes_importer.apply(
    'C:\PM contest\Common challenge\DomesticDeclarations.xes_\DomesticDeclarations.xes')

In [5]:
int_declarations = xes_importer.apply(
    'C:\PM contest\Common challenge\InternationalDeclarations.xes_\InternationalDeclarations.xes')

In [6]:
permit_log = xes_importer.apply(
    'C:\PM contest\Common challenge\PermitLog.xes_\PermitLog.xes')

In [7]:
prepaid_travel_cost = xes_importer.apply(
    'C:\PM contest\Common challenge\PrepaidTravelCost.xes_\PrepaidTravelCost.xes')

In [8]:
request_for_payment = xes_importer.apply(
    'C:\PM contest\Common challenge\RequestForPayment.xes_\RequestForPayment.xes')

In [5]:
domestic_declarations = to_dataframe.apply(domestic_declarations)
#int_declarations = to_dataframe.apply(int_declarations)
#permit_log = to_dataframe.apply(permit_log)
#prepaid_travel_cost = to_dataframe.apply(prepaid_travel_cost)
#request_for_payment = to_dataframe.apply(request_for_payment)

## Domestic Decararions & IDs

In [10]:
# проверить совпадают ли id с айди деклараций

In [11]:
domestic_declarations['id_num'] = domestic_declarations['id'].map(lambda x: int(x.split()[-1].split('_')[0]))
domestic_declarations['case:id_num'] = domestic_declarations['case:id'].map(lambda x: int(x.split()[-1]))
domestic_declarations['ccn_num'] = domestic_declarations['case:id'].map(lambda x: int(x.split()[-1]))

In [12]:
domestic_declarations['id_vs_case:id'] = domestic_declarations['id_num']-domestic_declarations['case:id_num']
# case concept name совпадает с case:id
domestic_declarations['ccn_vs_case:id'] = domestic_declarations['ccn_num']-domestic_declarations['case:id_num']
domestic_declarations['id_vs_ccn'] = domestic_declarations['id_num']-domestic_declarations['ccn_num']

In [13]:
len(domestic_declarations)

56437

In [14]:
len(domestic_declarations[domestic_declarations['id_vs_case:id']==0])

20084

In [15]:
# на исходную
domestic_declarations = xes_importer.apply(
    'C:\PM contest\Common challenge\DomesticDeclarations.xes_\DomesticDeclarations.xes')
domestic_declarations = to_dataframe.apply(domestic_declarations)

In [16]:
domestic_declarations.head(3)

,id,org:resource,concept:name,time:timestamp,org:role,case:id,case:concept:name,case:BudgetNumber,case:DeclarationNumber,case:Amount
0,st_step 86794_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-01-09 09:49:50+01:00,EMPLOYEE,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205
1,st_step 86793_0,STAFF MEMBER,Declaration FINAL_APPROVED by SUPERVISOR,2017-01-09 11:27:48+01:00,SUPERVISOR,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205
2,dd_declaration 86791_19,SYSTEM,Request Payment,2017-01-10 09:34:44+01:00,UNDEFINED,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205


In [17]:
domestic_declarations['d_id'] = domestic_declarations['case:id'].map(lambda x: int(x.split()[-1]))

In [18]:
ids_ = domestic_declarations.groupby(['d_id'])[['case:id']].count().sort_values(['case:id'], ascending = False)

In [19]:
domestic_declarations['id_param[0]'] = domestic_declarations['id'].map(lambda x: x.split()[0])
domestic_declarations['id_step'] = domestic_declarations['id'].map(lambda x: x.split()[-1].split('_')[-1])

In [20]:
domestic_declarations.head(2)

,id,org:resource,concept:name,time:timestamp,org:role,case:id,case:concept:name,case:BudgetNumber,case:DeclarationNumber,case:Amount,d_id,id_param[0],id_step
0,st_step 86794_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-01-09 09:49:50+01:00,EMPLOYEE,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,86791,st_step,0
1,st_step 86793_0,STAFF MEMBER,Declaration FINAL_APPROVED by SUPERVISOR,2017-01-09 11:27:48+01:00,SUPERVISOR,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,86791,st_step,0


In [21]:
domestic_declarations['id_param[0]'].value_counts()

st_step           36353
dd_declaration    20084
Name: id_param[0], dtype: int64

In [22]:
# таблица, в которой мы можем отследить оплаченные декларации
domestic_declarations.groupby(['d_id','id_param[0]'])[['id']].count()

id
d_id   id_param[0]       
86564  dd_declaration   2
       st_step          5
86572  dd_declaration   2
       st_step          2
86576  dd_declaration   2
...                    ..
143451 dd_declaration   2
       st_step          3
143456 dd_declaration   2
       st_step          4
143462 st_step          3

[20547 rows x 1 columns]

In [23]:
# можем посмотреть, по каким заявкам были запросы на оплату, на какие оплата проведена
domestic_declarations[domestic_declarations['id_param[0]']=='dd_declaration'].groupby(['d_id','concept:name'])[['id']].count()

id
d_id   concept:name       
86564  Payment Handled   1
       Request Payment   1
86572  Payment Handled   1
       Request Payment   1
86576  Payment Handled   1
...                     ..
143446 Request Payment   1
143451 Payment Handled   1
       Request Payment   1
143456 Payment Handled   1
       Request Payment   1

[20084 rows x 1 columns]

In [24]:
domestic_declarations[domestic_declarations['d_id']==137336].sort_values(['time:timestamp'], ascending = True)

,id,org:resource,concept:name,time:timestamp,org:role,case:id,case:concept:name,case:BudgetNumber,case:DeclarationNumber,case:Amount,d_id,id_param[0],id_step
52112,st_step 137350_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2018-11-30 16:29:16+01:00,EMPLOYEE,declaration 137336,declaration 137336,budget 86566,declaration number 137337,77.102908,137336,st_step,0
52113,st_step 137349_0,STAFF MEMBER,Declaration APPROVED by ADMINISTRATION,2018-11-30 16:29:20+01:00,ADMINISTRATION,declaration 137336,declaration 137336,budget 86566,declaration number 137337,77.102908,137336,st_step,0
52114,st_step 137351_0,STAFF MEMBER,Declaration REJECTED by BUDGET OWNER,2018-12-03 09:29:24+01:00,BUDGET OWNER,declaration 137336,declaration 137336,budget 86566,declaration number 137337,77.102908,137336,st_step,0
52115,st_step 137352_0,STAFF MEMBER,Declaration REJECTED by EMPLOYEE,2018-12-04 00:59:24+01:00,EMPLOYEE,declaration 137336,declaration 137336,budget 86566,declaration number 137337,77.102908,137336,st_step,0
52116,st_step 137346_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2018-12-04 08:42:48+01:00,EMPLOYEE,declaration 137336,declaration 137336,budget 86566,declaration number 137337,77.102908,137336,st_step,0
52117,st_step 137347_0,STAFF MEMBER,Declaration REJECTED by ADMINISTRATION,2018-12-04 08:42:50+01:00,ADMINISTRATION,declaration 137336,declaration 137336,budget 86566,declaration number 137337,77.102908,137336,st_step,0
52118,st_step 137348_0,STAFF MEMBER,Declaration REJECTED by EMPLOYEE,2018-12-05 11:20:20+01:00,EMPLOYEE,declaration 137336,declaration 137336,budget 86566,declaration number 137337,77.102908,137336,st_step,0
52119,st_step 137343_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2018-12-05 11:46:26+01:00,EMPLOYEE,declaration 137336,declaration 137336,budget 86566,declaration number 137337,77.102908,137336,st_step,0
52120,st_step 137345_0,STAFF MEMBER,Declaration REJECTED by ADMINISTRATION,2018-12-05 11:46:28+01:00,ADMINISTRATION,declaration 137336,declaration 137336,budget 86566,declaration number 137337,77.102908,137336,st_step,0
52121,st_step 137344_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2018-12-06 10:32:02+01:00,EMPLOYEE,declaration 137336,declaration 137336,budget 86566,declaration number 137337,77.102908,137336,st_step,0


## Permit log & ID

In [25]:
len(permit_log.columns)

173

Необходимо провести аггрегацию по избыточным столбцам:
- case:dec_id
- case:DeclarationNumber
- case:RequestedAmount
- case:Activity 
- case:RfpNumber
- case:OrganizationalEntity
- case:Rfp_id (Request For Payment)
- case:CostType
- case:Task
- case:Project

### aggregation dec_id

In [26]:
declaration_columns = list(permit_log.filter(regex=("case:dec_id.*")).columns)

In [27]:
permit_log['gen_dec_id'] = None

In [28]:
import numpy as np
for column in declaration_columns:
    permit_log['gen_dec_id'] = np.where(permit_log['gen_dec_id'].isna(),
                                        permit_log[column],permit_log['gen_dec_id'])  

In [29]:
permit_log[permit_log['gen_dec_id'].isna()].head(4)

,id,org:resource,concept:name,time:timestamp,org:role,case:OrganizationalEntity,case:ProjectNumber,case:TaskNumber,case:dec_id_0,case:ActivityNumber,...,case:Cost Type_10,case:Cost Type_11,case:Cost Type_12,case:Task_5,case:Task_4,case:Task_9,case:Task_8,case:Task_7,case:Task_6,gen_dec_id
24,rv_travel permit 76734_6,STAFF MEMBER,Start trip,2016-12-12 00:00:00+01:00,EMPLOYEE,organizational unit 65458,UNKNOWN,UNKNOWN,NaN,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,rv_travel permit 76734_7,STAFF MEMBER,End trip,2017-01-02 00:00:00+01:00,EMPLOYEE,organizational unit 65458,UNKNOWN,UNKNOWN,NaN,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,st_step 76735_0,STAFF MEMBER,Permit SAVED by EMPLOYEE,2017-02-07 16:08:44+01:00,EMPLOYEE,organizational unit 65458,UNKNOWN,UNKNOWN,NaN,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,st_step 76681_0,STAFF MEMBER,Permit SUBMITTED by EMPLOYEE,2017-01-09 11:45:18+01:00,EMPLOYEE,organizational unit 65458,UNKNOWN,UNKNOWN,NaN,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### aggregation DeclarationNumber 

In [30]:
declaration_nums = list(permit_log.filter(regex=("case:DeclarationNumber.*")).columns)

In [31]:
permit_log['gen_dec_num'] = None

In [32]:
for column in declaration_nums:
    permit_log['gen_dec_num'] = np.where(permit_log['gen_dec_num'].isna(),
                                         permit_log[column],permit_log['gen_dec_num'])  

In [33]:
permit_log[permit_log['gen_dec_num'].isna()].head()

,id,org:resource,concept:name,time:timestamp,org:role,case:OrganizationalEntity,case:ProjectNumber,case:TaskNumber,case:dec_id_0,case:ActivityNumber,...,case:Cost Type_11,case:Cost Type_12,case:Task_5,case:Task_4,case:Task_9,case:Task_8,case:Task_7,case:Task_6,gen_dec_id,gen_dec_num
24,rv_travel permit 76734_6,STAFF MEMBER,Start trip,2016-12-12 00:00:00+01:00,EMPLOYEE,organizational unit 65458,UNKNOWN,UNKNOWN,NaN,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,rv_travel permit 76734_7,STAFF MEMBER,End trip,2017-01-02 00:00:00+01:00,EMPLOYEE,organizational unit 65458,UNKNOWN,UNKNOWN,NaN,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,st_step 76735_0,STAFF MEMBER,Permit SAVED by EMPLOYEE,2017-02-07 16:08:44+01:00,EMPLOYEE,organizational unit 65458,UNKNOWN,UNKNOWN,NaN,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,st_step 76681_0,STAFF MEMBER,Permit SUBMITTED by EMPLOYEE,2017-01-09 11:45:18+01:00,EMPLOYEE,organizational unit 65458,UNKNOWN,UNKNOWN,NaN,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,st_step 76682_0,STAFF MEMBER,Permit FINAL_APPROVED by SUPERVISOR,2017-01-09 11:45:21+01:00,SUPERVISOR,organizational unit 65458,UNKNOWN,UNKNOWN,NaN,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### aggregation RequestedAmount

In [34]:
req_amount_nums = list(permit_log.filter(regex=("case:RequestedAmount.*")).columns)

In [35]:
permit_log['gen_req_amount'] = None

In [36]:
for column in req_amount_nums:
    permit_log['gen_req_amount'] = np.where(permit_log['gen_req_amount'].isna(),
                                            permit_log[column],permit_log['gen_req_amount'])  

### aggregation Activity

In [37]:
activity_list = list(permit_log.filter(regex=("case:Activity.*")).columns)

In [38]:
permit_log['gen_activity'] = None

In [39]:
for column in activity_list:
    permit_log['gen_activity'] = np.where(permit_log['gen_activity'].isna(),
                                          permit_log[column],permit_log['gen_activity'])  

### aggregation RfpNumber

In [40]:
rfpn_list = list(permit_log.filter(regex=("case:RfpNumber.*")).columns)

In [41]:
permit_log['gen_rfpn'] = None

In [42]:
for column in rfpn_list:
    permit_log['gen_rfpn'] = np.where(permit_log['gen_rfpn'].isna(),
                                      permit_log[column],permit_log['gen_rfpn'])  

### aggregation OrganisationalEntity

In [43]:
oe_list = list(permit_log.filter(regex=("case:OrganizationalEntity.*")).columns)

In [44]:
permit_log['gen_org_ent'] = None

In [45]:
for column in oe_list:
    permit_log['gen_org_ent'] = np.where(permit_log['gen_org_ent'].isna(),
                                         permit_log[column],permit_log['gen_org_ent'])  

### aggregation Rfp_id

In [46]:
rfp_ids = list(permit_log.filter(regex=("case:Rfp_id.*")).columns)

In [47]:
permit_log['gen_rfp_id'] = None

In [48]:
for column in rfp_ids:
    permit_log['gen_rfp_id'] = np.where(permit_log['gen_rfp_id'].isna(),
                                        permit_log[column],permit_log['gen_rfp_id'])  

### aggregation CostType

In [49]:
cost_types = list(permit_log.filter(regex=("case:Cost Type.*")).columns)

In [50]:
permit_log['gen_cost_type'] = None

In [51]:
for column in cost_types:
    permit_log['gen_cost_type'] = np.where(permit_log['gen_cost_type'].isna(),
                                           permit_log[column],permit_log['gen_cost_type'])  

### aggregation Task

In [52]:
task_list = list(permit_log.filter(regex=("case:Task.*")).columns)

In [53]:
permit_log['gen_tasks'] = None

In [54]:
for column in task_list:
    permit_log['gen_tasks'] = np.where(permit_log['gen_tasks'].isna(),
                                           permit_log[column],permit_log['gen_tasks'])  

### aggregation Project

In [55]:
project_list = list(permit_log.filter(regex=("case:Project.*")).columns)

In [56]:
permit_log['gen_projects'] = None

In [57]:
for column in project_list:
    permit_log['gen_projects'] = np.where(permit_log['gen_projects'].isna(),
                                           permit_log[column],permit_log['gen_projects'])  

## Delete all extra columns

In [58]:
permit_log = permit_log.drop(
    declaration_nums+declaration_columns+req_amount_nums+activity_list+rfpn_list+oe_list+rfp_ids+cost_types+task_list+project_list,
    axis = 1)

In [59]:
len(permit_log.columns)

23

In [60]:
print(*permit_log.columns)

id org:resource concept:name time:timestamp org:role case:TotalDeclared case:concept:name case:Overspent case:travel permit number case:id case:RequestedBudget case:BudgetNumber case:OverspentAmount gen_dec_id gen_dec_num gen_req_amount gen_activity gen_rfpn gen_org_ent gen_rfp_id gen_cost_type gen_tasks gen_projects


In [61]:
# Ура, теперь здесь 23 столбца вместо 170 

In [62]:
# с cost_type нужно быть аккуратнее, потому что там только нолики - бинарное кодирование (?)
permit_log[['gen_dec_id', 'gen_dec_num', 'gen_req_amount', 'gen_activity', 'gen_rfpn', 'gen_org_ent',
           'gen_rfp_id', 'gen_cost_type', 'gen_tasks', 'gen_projects']]

,gen_dec_id,gen_dec_num,gen_req_amount,gen_activity,gen_rfpn,gen_org_ent,gen_rfp_id,gen_cost_type,gen_tasks,gen_projects
0,declaration 76457,declaration number 76458,56,activity 46005,NaN,organizational unit 65458,NaN,NaN,UNKNOWN,UNKNOWN
1,declaration 76457,declaration number 76458,56,activity 46005,NaN,organizational unit 65458,NaN,NaN,UNKNOWN,UNKNOWN
2,declaration 76457,declaration number 76458,56,activity 46005,NaN,organizational unit 65458,NaN,NaN,UNKNOWN,UNKNOWN
3,declaration 76457,declaration number 76458,56,activity 46005,NaN,organizational unit 65458,NaN,NaN,UNKNOWN,UNKNOWN
4,declaration 76457,declaration number 76458,56,activity 46005,NaN,organizational unit 65458,NaN,NaN,UNKNOWN,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...
86576,NaN,NaN,NaN,UNKNOWN,NaN,organizational unit 65464,NaN,NaN,task 427,project 13110
86577,NaN,NaN,NaN,UNKNOWN,NaN,organizational unit 65464,NaN,NaN,task 427,project 13110
86578,NaN,NaN,NaN,UNKNOWN,NaN,organizational unit 65464,NaN,NaN,task 427,project 13110
86579,NaN,NaN,NaN,UNKNOWN,NaN,organizational unit 65464,NaN,NaN,task 427,project 13110


In [63]:
# человеческий permit_log
permit_log.columns

Index(['id', 'org:resource', 'concept:name', 'time:timestamp', 'org:role',
       'case:TotalDeclared', 'case:concept:name', 'case:Overspent',
       'case:travel permit number', 'case:id', 'case:RequestedBudget',
       'case:BudgetNumber', 'case:OverspentAmount', 'gen_dec_id',
       'gen_dec_num', 'gen_req_amount', 'gen_activity', 'gen_rfpn',
       'gen_org_ent', 'gen_rfp_id', 'gen_cost_type', 'gen_tasks',
       'gen_projects'],
      dtype='object')

In [64]:
# переименовала из временных названий в общие
permit_log.columns = ['id', 'org:resource', 'concept:name', 'time:timestamp', 'org:role',
       'case:TotalDeclared', 'case:concept:name', 'case:Overspent',
       'case:travel permit number', 'case:id', 'case:RequestedBudget',
       'case:BudgetNumber', 'case:OverspentAmount','case:dec_id',
       'case:DeclarationNumber','case:RequestedAmount','case:Activity',
       'case:RfpNumber','case:OrganizationalEntity','case:Rfp_id','case:CostType',
       'case:Task','case:Project']

In [65]:
permit_log['case:RfpNumber']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
86576    NaN
86577    NaN
86578    NaN
86579    NaN
86580    NaN
Name: case:RfpNumber, Length: 86581, dtype: object

In [66]:
permit_log.to_csv('compact_PermitLog.csv')